In [23]:
%pip -q install google-genai

In [24]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [25]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Instagram, LinkedIn, etc.
*   **Assinar a newsletter da Alura:** Geralmente, eles anunciam os eventos por e-mail.
*   **Ficar de olho na página de eventos da Alura:** Verifique regularmente a seção de eventos do site da Alura.

Assim que a data for definida, a Alura certamente fará um grande anúncio!

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As aulas serão online e gratuitas e, no final, você receberá um certificado Alura + Google. As inscrições podem ser feitas até o dia 11 de maio.

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [26]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [27]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [28]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [29]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [30]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca notícias sobre o tópico indicado",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
      para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.

      Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.

      Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
      e pode ser substituído por outro que tenha mais.

      Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
      """
    )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  lancamentos = call_agent(buscador, entrada_do_agente_buscador)

  return lancamentos

In [31]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planjeador de conteúdo, especialista em redes sociais. Com base na lista
        de lançamentos mais recentes, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontar mais
        informações sobre os temas e aprofundar.

        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas
        pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os
        assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [35]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
        tools=[google_search]
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [36]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
        tools=[google_search]
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [38]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Digite o tópico que deseja gerar o post.")
else:
  print(f"Vamos começar criando o post sobre as novidades em {topico}!")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado do Agente buscador ---\n")
  display(to_markdown(lancamentos_buscados))
  print("----------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do Agente Planejador ---\n")
  display(to_markdown(plano_de_post))
  print("----------------------------------------------------")

  rascunho_gerado = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do Agente Redator ---\n")
  display(to_markdown(rascunho_gerado))
  print("----------------------------------------------------")

  texto_revisado = agente_revisor(topico, rascunho_gerado)
  print("\n--- Resultado do Agente Revisor ---\n")
  display(to_markdown(texto_revisado))
  print("----------------------------------------------------")

  print("\n🎉 Post criado com sucesso! 🎉")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: NASA Astronomia
Vamos começar criando o post sobre as novidades em NASA Astronomia!

--- Resultado do Agente buscador ---



> Aqui estão alguns lançamentos recentes e relevantes da NASA em astronomia, considerando os últimos meses até 17 de maio de 2025:
> 
> 
> Com base nas notícias e lançamentos recentes da NASA em astronomia até 17 de maio de 2025, aqui estão alguns destaques relevantes:
> 
> 1.  **SPHEREx (Spectro-Photometer for the History of the Universe, Epoch of Reionization and Ices Explorer)**: Lançado em fevereiro de 2025, este observatório está mapeando o universo na luz do infravermelho próximo, coletando dados de mais de 450 milhões de galáxias e 100 milhões de estrelas na Via Láctea. A missão tem como objetivo estudar a formação de galáxias, gelo de água e materiais orgânicos em berçários estelares.
> 2.  **Lucy Mission**: Em abril de 2025, a missão Lucy realizou um sobrevoo do asteroide 52246 Donaldjohanson, capturando imagens detalhadas. A missão continua sua jornada para explorar os asteroides troianos de Júpiter, fornecendo dados valiosos sobre a composição e as características da superfície desses corpos celestes antigos.
> 3.  **James Webb Space Telescope (JWST) Discoveries**: O JWST continua a fazer descobertas notáveis. Em abril de 2025, foram divulgadas imagens detalhadas da nebulosa planetária NGC 1514, revelando nuances e anéis de poeira "difusos". O telescópio também descobriu evidências de um buraco negro supermassivo no coração da galáxia espiral Messier 83 (M83). Adicionalmente, o JWST observou detalhes das auroras de Júpiter e identificou duas estrelas responsáveis por gerar poeira rica em carbono na Via Láctea.
> 4.  **CLPS (Commercial Lunar Payload Services) Missions**: A NASA continua utilizando o programa CLPS para entregar cargas científicas e tecnológicas à Lua usando landers comerciais. Essas missões visam estudar a geologia lunar, testar novas tecnologias para futuras missões humanas e coletar dados sobre o ambiente lunar.
> 5.  **Europa Clipper**: A missão Europa Clipper, lançada em outubro de 2024, progrediu em sua jornada para a lua Europa de Júpiter. Em maio de 2025, a sonda capturou Marte em infravermelho. A chegada em Júpiter está prevista para 2030.
> 
> Outros pontos notáveis:
> 
> *   A missão Tianwen-2 da China, com lançamento previsto para maio de 2025, tem como objetivo coletar amostras de um asteroide próximo da Terra e estudar um cometa.
> *   A missão Hera da ESA realizou um sobrevoo de Marte em março de 2025 como parte da missão Asteroid Impact and Deflection Assessment.
> 
> Esses lançamentos e descobertas destacam o contínuo avanço da NASA na exploração espacial e na compreensão do universo.
> 


----------------------------------------------------

--- Resultado do Agente Planejador ---



> Okay, com base nos lançamentos recentes da NASA e outras missões espaciais até maio de 2025, vou elaborar um plano de conteúdo para posts em redes sociais. Para isso, usarei a ferramenta de busca para encontrar mais informações relevantes sobre cada tema.
> 
> 
> Com base nas pesquisas, aqui está um plano de conteúdo detalhado com foco no tema que parece mais interessante e relevante para um público de redes sociais:
> 
> **Tema Escolhido: James Webb Space Telescope (JWST) - Descoberta de Buraco Negro Supermassivo na Galáxia M83**
> 
> Este tema é particularmente atraente porque combina a grandiosidade dos buracos negros com as capacidades inovadoras do Telescópio Espacial James Webb, gerando grande impacto e curiosidade.
> 
> **Pontos Relevantes:**
> 
> *   **Descoberta Inédita:** O JWST revelou evidências de um buraco negro supermassivo no coração da galáxia Messier 83 (M83), algo que intrigava os astrônomos há décadas.
> *   **Galáxia M83:** Conhecida como a "Galáxia do Cata-Vento do Sul", está localizada a 15 milhões de anos-luz da Terra.
> *   **Núcleo Galáctico Ativo (AGN):** A descoberta sugere a presença de um AGN, uma região compacta e luminosa no centro da galáxia, alimentada pela matéria que cai no buraco negro.
> *   **Gás Neon Ionizado:** O JWST detectou emissões de gás neon ionizado, indicando a presença do buraco negro.
> *   **Importância da Descoberta:** A descoberta auxilia no entendimento de como buracos negros moldam o universo.
> *   **Tecnologia do JWST:** A capacidade infravermelha do JWST permitiu "ver" através da poeira cósmica, revelando o que estava escondido.
> *   **Próximos Passos:** Astrônomos planejam usar outros telescópios, como o ALMA e o VLT, para confirmar e estudar o buraco negro em detalhes.
> 
> **Plano de Conteúdo para Redes Sociais:**
> 
> 1.  **Título Impactante:**
>     *   "JWST Desvenda Mistério Cósmico: Buraco Negro 'Escondido' Revelado em Galáxia Distante!"
>     *   "Telescópio James Webb Faz Descoberta Histórica: Um Buraco Negro Supermassivo na M83!"
> 
> 2.  **Conteúdo do Post:**
> 
>     *   **Introdução:**
>         *   "Prepare-se para uma descoberta que vai expandir seus horizontes cósmicos! O Telescópio Espacial James Webb (JWST) acaba de revelar evidências de um buraco negro supermassivo no coração da galáxia Messier 83 (M83), um mistério que intrigava astrônomos há décadas!"
>     *   **Detalhes da Descoberta:**
>         *   "Localizada a 15 milhões de anos-luz de nós, a M83, também conhecida como a 'Galáxia do Cata-Vento do Sul', sempre foi peculiar. Agora, o JWST detectou emissões de gás neon ionizado, um forte indício de um Núcleo Galáctico Ativo (AGN) alimentado por um buraco negro faminto!"
>     *   **Por que Isso Importa:**
>         *   "Essa descoberta não é apenas uma curiosidade cósmica. Ela nos ajuda a entender melhor como os buracos negros supermassivos influenciam a evolução das galáxias e moldam o universo como o conhecemos!"
>     *   **O Poder do JWST:**
>         *   "O JWST, com sua incrível capacidade de 'ver' através da poeira cósmica usando luz infravermelha, conseguiu onde outros falharam. É como ter óculos de visão noturna para o universo profundo!"
>     *   **O Que Vem a Seguir:**
>         *   "A saga não termina aqui! Os cientistas agora usarão outros telescópios poderosos, como o ALMA e o VLT, para confirmar e estudar esse buraco negro em detalhes. Fiquem ligados para mais atualizações!"
>     *   **Chamada para Ação:**
>         *   "O que você acha dessa descoberta? Compartilhe suas teorias e perguntas nos comentários! E não se esqueça de seguir nossa página para mais notícias incríveis do universo da astronomia!"
> 
> 3.  **Elementos Visuais:**
> 
>     *   **Imagem:** Utilizar a imagem mais recente e impressionante da galáxia M83 obtida pelo JWST.
>     *   **Vídeo (opcional):** Um vídeo curto explicando o que são buracos negros supermassivos e como o JWST os detecta.
> 
> 4.  **Hashtags:**
> 
>     *   #JWST #JamesWebb #BuracoNegro #M83 #Astronomia #Espaço #Ciência #Descoberta #Galáxia #Universo #NASA
> 
> 5.  **Adaptações para Diferentes Plataformas:**
> 
>     *   **Twitter:** Posts mais concisos, focando nos highlights da descoberta.
>     *   **Instagram:** Ênfase na imagem visualmente atraente, com legendas informativas.
>     *   **Facebook:** Conteúdo mais detalhado e explicativo, incentivando a discussão.
>     *   **TikTok:** Vídeos curtos e dinâmicos explicando a descoberta de forma simples e envolvente.
> 
> Este plano de conteúdo busca não apenas informar, mas também inspirar e engajar o público, mostrando a importância e a beleza da exploração espacial e das descobertas científicas.
> 


----------------------------------------------------

--- Resultado do Agente Redator ---



> 🔭🌌 **IMAGEM:** Uma foto deslumbrante da galáxia M83 capturada pelo James Webb.
> 
> Prepare-se para ter sua mente explodida! 🤯 O Telescópio Espacial James Webb acaba de fazer uma descoberta INACREDITÁVEL: um buraco negro supermassivo escondido no coração da galáxia M83! 😱
> 
> Localizada a 15 milhões de anos-luz de nós, a M83, também conhecida como a "Galáxia do Cata-Vento do Sul", guardava um segredo cósmico. O JWST detectou emissões de gás neon ionizado, revelando um Núcleo Galáctico Ativo (AGN) faminto, alimentado por um buraco negro gigante! 🕳️✨
> 
> E por que isso importa? 🤔 Essa descoberta nos ajuda a entender como esses monstros cósmicos moldam as galáxias e o universo! O JWST, com sua visão infravermelha superpoderosa, conseguiu enxergar através da poeira cósmica e revelar o que estava escondido. 🔭
> 
> E não para por aí! 🚀 Os cientistas vão usar outros telescópios para estudar esse buraco negro de perto. Fique ligado para mais atualizações cósmicas! 😉
> 
> O que você acha dessa descoberta? Compartilhe sua opinião nos comentários e marque um amigo que ama astronomia! 👇
> 
> #JWST #BuracoNegro #Astronomia #Alura 
> 


----------------------------------------------------

--- Resultado do Agente Revisor ---



> O rascunho está bom, mas podemos fazer algumas melhorias para deixá-lo ainda mais atraente e informativo para o público do Instagram:
> 
> **Problemas e Sugestões:**
> 
> 1.  **Tom:** O tom está bom, mas podemos equilibrar o entusiasmo com um pouco mais de informação acessível.
> 2.  **Emojis:** O uso excessivo de emojis pode ser um pouco demais. Sugiro reduzir um pouco para não sobrecarregar a leitura.
> 3.  **Hashtags:** Adicionar algumas hashtags mais específicas pode aumentar o alcance do post.
> 
> **Rascunho Revisado:**
> 
> 🔭🌌 **IMAGEM:** Uma foto deslumbrante da galáxia M83 capturada pelo James Webb.
> 
> 💥 Preparados para essa? O Telescópio Espacial James Webb acaba de revelar um segredo cósmico na galáxia M83: um buraco negro supermassivo!
> 
> Localizada a 15 milhões de anos-luz, a M83 (ou Galáxia do Cata-Vento do Sul) escondia um Núcleo Galáctico Ativo (AGN). O JWST detectou emissões de gás neon ionizado, mostrando que um buraco negro gigante está se alimentando ali! 🕳️✨
> 
> Por que isso é tão incrível? 🤔 Entender esses "monstros cósmicos" nos ajuda a descobrir como as galáxias e o universo evoluem! Com sua visão infravermelha superpoderosa, o JWST consegue enxergar através da poeira cósmica e revelar segredos como esse. 🔭
> 
> E tem mais! 🚀 Os cientistas agora vão usar outros telescópios para estudar esse buraco negro de pertinho. Fiquem ligados para mais novidades do universo! 😉
> 
> O que acharam dessa descoberta? Comentem aqui embaixo e marquem um amigo que curte astronomia! 👇
> 
> #JWST #BuracoNegro #Astronomia #M83 #JamesWebbTelescope #Astrofísica #Galáxias #Alura
> 


----------------------------------------------------

🎉 Post criado com sucesso! 🎉
